## A. Configuración general.

In [ ]:
#1. Librerías.
%run "../librerias.ipynb"

Matplotlib is building the font cache; this may take a moment.


/home/ianlink95/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/ianlink95/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#2. Constantes.
%run "../constantes.ipynb"

dataset_input = dataset_file_preprocesado_24_2_DQ
dataset_output = dataset_file_fe_24_2_DQ_eug
ventana = 2

In [ ]:
#3. Funciones.
%run "../funciones.ipynb"

In [4]:
#4. Configuración de jupysql.
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [5]:
#5. Conexión a DuckDB.
%sql duckdb:///:memory:

In [6]:
#6. Lectura de datos.

In [ ]:
%%sql
--ROLLBACK;
create or replace table competencia_02 as
select
    *
from read_csv_auto("{{dataset_input}}")

,Success


## B. Creación de variables intramensuales.

In [ ]:
#1. Normalizo ctrx_quarter según la antigüedad del cliente.

In [ ]:
%%sql
create or replace table competencia_02 as
select *,
    CASE 
        WHEN cliente_antiguedad = 1 THEN ctrx_quarter * 5
        WHEN cliente_antiguedad = 2 THEN ctrx_quarter * 2
        WHEN cliente_antiguedad = 3 THEN ctrx_quarter * 1.2
        ELSE ctrx_quarter
    END AS ctrx_quarter_normalizado,
from competencia_02;

,Success


In [ ]:
#2. Sumo columnas.

In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
    , mcuenta_corriente_adicional + mcuenta_corriente + mcaja_ahorro_adicional + mcaja_ahorro_dolares + mcaja_ahorro AS suma_alt # ninguna tiene null
    , cseguro_vida + cseguro_auto +	cseguro_vivienda + cseguro_accidentes_personales AS cseguro_todos # ninguna tiene null
    , ifnull(matm, 0) + ifnull(matm_other, 0) as matm_total
    , ifnull(Visa_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tarjeta_msaldototal
    , ifnull(mactivos_margen, 0) +  ifnull(mpasivos_margen, 0) as map_margen
    , ifnull(ccuenta_corriente, 0) +  ifnull(ccaja_ahorro, 0) as cc_ca
    , ifnull(ctarjeta_visa, 0) +  ifnull(ctarjeta_Visa, 0) as ctarjeta_total
    , ifnull(ctarjeta_visa_transacciones, 0) +  ifnull(ctarjeta_master_transacciones, 0) as ctarjeta_total_transacciones
    , ifnull(mtarjeta_visa_consumo, 0) +  ifnull(mtarjeta_master_consumo, 0) as mtarjeta_total_consumo
    , ifnull(mplazo_fijo_dolares, 0) +  ifnull(mplazo_fijo_pesos, 0) as mplazo_fijo
    , ifnull(minversion1_pesos, 0) +  ifnull(minversion1_dolares, 0) as minversion1
    , ifnull(mpayroll, 0) +  ifnull(mpayroll2, 0) as mpayroll_todos
    , ifnull(mcuenta_debitos_automaticos, 0) +  ifnull(mtarjeta_total_consumo, 0) as consumos_mas_DA
    , ifnull(ctarjeta_visa_debitos_automaticos, 0) +  ifnull(ctarjeta_master_debitos_automaticos, 0) as ctarjeta_todas_debitos_automaticos
    , ifnull(mttarjeta_visa_debitos_automaticos, 0) +  ifnull(mttarjeta_master_debitos_automaticos, 0) as mtarjeta_todas_debitos_automaticos
    , ifnull(ctarjeta_visa_descuentos, 0) +  ifnull(ctarjeta_master_descuentos, 0) as ctarjeta_todas_descuentos
    , ifnull(mtarjeta_visa_descuentos, 0) +  ifnull(mtarjeta_master_descuentos, 0) as mtarjeta_todas_descuentos
    , ifnull(ccomisiones_mantenimiento, 0) +  ifnull(ccomisiones_otras, 0) as comisiones
    , ifnull(mcomisiones_mantenimiento, 0) +  ifnull(mcomisiones_otras, 0) as mcomisiones_suma
    , ifnull(ctransferencias_recibidas, 0) +  ifnull(ctransferencias_emitidas, 0) as ctransferencias
    , ifnull(mtransferencias_recibidas, 0) +  ifnull(mtransferencias_emitidas, 0) as mtransferencias
    , ifnull(Master_msaldopesos, 0) +  ifnull(Visa_msaldopesos, 0) as tarjetas_msaldopesos
    , ifnull(Master_msaldodolares, 0) +  ifnull(Visa_msaldodolares, 0) as tarjetas_msaldodolares
    , ifnull(Master_mconsumospesos, 0) +  ifnull(Visa_mconsumospesos, 0) as tarjetas_mconsumospesos
    , ifnull(Master_mconsumosdolares, 0) +  ifnull(Visa_mconsumosdolares, 0) as tarjetas_mconsumosdolares
    , ifnull(Master_mlimitecompra, 0) +  ifnull(Visa_mlimitecompra, 0) as tarjetas_mlimitecompra
    , ifnull(Master_madelantopesos, 0) +  ifnull(Visa_madelantopesos, 0) as tarjetas_madelantopesos
    , ifnull(Master_madelantodolares, 0) +  ifnull(Visa_madelantodolares, 0) as tarjetas_madelantodolares
    , ifnull(Master_mpagado, 0) +  ifnull(Visa_mpagado, 0) as tarjetas_mpagado
    , ifnull(Master_mpagospesos, 0) +  ifnull(Visa_mpagospesos, 0) as tarjetas_mpagospesos
    , ifnull(Master_mpagosdolares, 0) +  ifnull(Visa_mpagosdolares, 0) as tarjetas_mpagosdolares
    , ifnull(Master_mconsumototal, 0) +  ifnull(Visa_mconsumototal, 0) as tarjetas_mconsumototal
    , ifnull(Master_cconsumos, 0) +  ifnull(Visa_cconsumos, 0) as tarjetas_cconsumos
    , ifnull(Master_cadelantosefectivo, 0) +  ifnull(Visa_cadelantosefectivo, 0) as tarjetas_cadelantosefectivo
    , ifnull(Master_mpagominimo, 0) +  ifnull(Visa_mpagominimo, 0) as tarjetas_mpagominimo
    , ifnull(Master_mfinanciacion_limite, 0) +  ifnull(Visa_mfinanciacion_limite, 0) as tarjetas_mfinanciacion_limite
from competencia_02

,Success


In [ ]:
#3. Saco mejores y peores fechas.

In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
    , greatest(Master_fechaalta, Visa_fechaalta) as tc_fechaalta_mayor
    , least(Master_fechaalta, Visa_fechaalta) as tc_fechalta_menor
    , greatest(Master_Finiciomora,Visa_Finiciomora) as tc_fechamora_mayor
    , least(Master_Finiciomora,Visa_Finiciomora) as tc_fechamora_menor
    , greatest(Master_fultimo_cierre,Visa_fultimo_cierre) as tc_fechacierre_mayor
    , least(Master_fultimo_cierre,Visa_fultimo_cierre) as tc_fechacierre_menor
from competencia_02;

,Success


In [ ]:
#4. Calculo Métricas de posición ante antigüedad, fechas de altas, etc.

In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6 # Corto la poblacion en 10, y le asigno una a cada cliente, según su posición de antigüedad.
from competencia_02
order by numero_de_cliente, cliente_antiguedad

,Success


In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
    ,ntile(10) over (partition by foto_mes order by tc_fechaalta_mayor) as antiguedad_tarjetas
from competencia_02
order by numero_de_cliente, tc_fechaalta_mayor

,Success


In [ ]:
#5. Calculo Promedios móviles con variables originales.

In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
    , avg(mrentabilidad) over ventana_2 as avg2_mrentabilidad
    , avg(mrentabilidad_annual) over ventana_2 as avg2_mrentabilidad_annual
    , avg(mcomisiones) over ventana_2 as avg2_mcomisiones
    , avg(mactivos_margen) over ventana_2 as avg2_mactivos_margen
    , avg(mpasivos_margen) over ventana_2 as avg2_mpasivos_margen
    , avg(cproductos) over ventana_2 as avg2_cproductos
    , avg(tcuentas) over ventana_2 as avg2_tcuentas
    , avg(ccuenta_corriente) over ventana_2 as avg2_ccuenta_corriente
    , avg(mcuenta_corriente_adicional) over ventana_2 as avg2_mcuenta_corriente_adicional
    , avg(mcuenta_corriente) over ventana_2 as avg2_mcuenta_corriente
    , avg(ccaja_ahorro) over ventana_2 as avg2_ccaja_ahorro
    , avg(mcaja_ahorro) over ventana_2 as avg2_mcaja_ahorro
    , avg(mcaja_ahorro_adicional) over ventana_2 as avg2_mcaja_ahorro_adicional
    , avg(mcaja_ahorro_dolares) over ventana_2 as avg2_mcaja_ahorro_dolares
    , avg(mcuentas_saldo) over ventana_2 as avg2_mcuentas_saldo
    , avg(ctarjeta_debito) over ventana_2 as avg2_ctarjeta_debito
    , avg(ctarjeta_debito_transacciones) over ventana_2 as avg2_ctarjeta_debito_transacciones
    , avg(mautoservicio) over ventana_2 as avg2_mautoservicio
    , avg(ctarjeta_visa) over ventana_2 as avg2_ctarjeta_visa
    , avg(ctarjeta_visa_transacciones) over ventana_2 as avg2_ctarjeta_visa_transacciones
    , avg(mtarjeta_visa_consumo) over ventana_2 as avg2_mtarjeta_visa_consumo
    , avg(ctarjeta_master) over ventana_2 as avg2_ctarjeta_master
    , avg(ctarjeta_master_transacciones) over ventana_2 as avg2_ctarjeta_master_transacciones
    , avg(mtarjeta_master_consumo) over ventana_2 as avg2_mtarjeta_master_consumo
    # , avg(cprestamos_personales) over ventana_2 as avg2_cprestamos_personales
    # , avg(mprestamos_personales) over ventana_2 as avg2_mprestamos_personales
    , avg(cprestamos_prendarios) over ventana_2 as avg2_cprestamos_prendarios
    , avg(mprestamos_prendarios) over ventana_2 as avg2_mprestamos_prendarios
    , avg(cprestamos_hipotecarios) over ventana_2 as avg2_cprestamos_hipotecarios
    , avg(mprestamos_hipotecarios) over ventana_2 as avg2_mprestamos_hipotecarios
    , avg(cplazo_fijo) over ventana_2 as avg2_cplazo_fijo
    , avg(mplazo_fijo_dolares) over ventana_2 as avg2_mplazo_fijo_dolares
    , avg(mplazo_fijo_pesos) over ventana_2 as avg2_mplazo_fijo_pesos
    , avg(cinversion1) over ventana_2 as avg2_cinversion1
    , avg(minversion1_pesos) over ventana_2 as avg2_minversion1_pesos
    , avg(minversion1_dolares) over ventana_2 as avg2_minversion1_dolares
    , avg(cinversion2) over ventana_2 as avg2_cinversion2
    , avg(minversion2) over ventana_2 as avg2_minversion2
    , avg(cseguro_vida) over ventana_2 as avg2_cseguro_vida
    , avg(cseguro_auto) over ventana_2 as avg2_cseguro_auto
    , avg(cseguro_vivienda) over ventana_2 as avg2_cseguro_vivienda
    , avg(cseguro_accidentes_personales) over ventana_2 as avg2_cseguro_accidentes_personales
    , avg(cpayroll_trx) over ventana_2 as avg2_cpayroll_trx
    , avg(mpayroll) over ventana_2 as avg2_mpayroll
    , avg(mpayroll2) over ventana_2 as avg2_mpayroll2
    , avg(cpayroll2_trx) over ventana_2 as avg2_cpayroll2_trx
    , avg(ccuenta_debitos_automaticos) over ventana_2 as avg2_ccuenta_debitos_automaticos
    , avg(mcuenta_debitos_automaticos) over ventana_2 as avg2_mcuenta_debitos_automaticos
    , avg(ctarjeta_visa_debitos_automaticos) over ventana_2 as avg2_ctarjeta_visa_debitos_automaticos
    , avg(mttarjeta_visa_debitos_automaticos) over ventana_2 as avg2_mttarjeta_visa_debitos_automaticos
    , avg(ctarjeta_master_debitos_automaticos) over ventana_2 as avg2_ctarjeta_master_debitos_automaticos
    , avg(mttarjeta_master_debitos_automaticos) over ventana_2 as avg2_mttarjeta_master_debitos_automaticos
    , avg(cpagodeservicios) over ventana_2 as avg2_cpagodeservicios
    , avg(mpagodeservicios) over ventana_2 as avg2_mpagodeservicios
    , avg(cpagomiscuentas) over ventana_2 as avg2_cpagomiscuentas
    , avg(mpagomiscuentas) over ventana_2 as avg2_mpagomiscuentas
    , avg(ccajeros_propios_descuentos) over ventana_2 as avg2_ccajeros_propios_descuentos
    , avg(mcajeros_propios_descuentos) over ventana_2 as avg2_mcajeros_propios_descuentos
    , avg(ctarjeta_visa_descuentos) over ventana_2 as avg2_ctarjeta_visa_descuentos
    , avg(mtarjeta_visa_descuentos) over ventana_2 as avg2_mtarjeta_visa_descuentos
    , avg(ctarjeta_master_descuentos) over ventana_2 as avg2_ctarjeta_master_descuentos
    , avg(mtarjeta_master_descuentos) over ventana_2 as avg2_mtarjeta_master_descuentos
    , avg(ccomisiones_mantenimiento) over ventana_2 as avg2_ccomisiones_mantenimiento
    , avg(mcomisiones_mantenimiento) over ventana_2 as avg2_mcomisiones_mantenimiento
    , avg(ccomisiones_otras) over ventana_2 as avg2_ccomisiones_otras
    , avg(mcomisiones_otras) over ventana_2 as avg2_mcomisiones_otras
    , avg(cforex) over ventana_2 as avg2_cforex
    , avg(cforex_buy) over ventana_2 as avg2_cforex_buy
    , avg(mforex_buy) over ventana_2 as avg2_mforex_buy
    , avg(cforex_sell) over ventana_2 as avg2_cforex_sell
    , avg(mforex_sell) over ventana_2 as avg2_mforex_sell
    , avg(ctransferencias_recibidas) over ventana_2 as avg2_ctransferencias_recibidas
    , avg(mtransferencias_recibidas) over ventana_2 as avg2_mtransferencias_recibidas
    , avg(ctransferencias_emitidas) over ventana_2 as avg2_ctransferencias_emitidas
    , avg(mtransferencias_emitidas) over ventana_2 as avg2_mtransferencias_emitidas
    , avg(cextraccion_autoservicio) over ventana_2 as avg2_cextraccion_autoservicio
    , avg(mextraccion_autoservicio) over ventana_2 as avg2_mextraccion_autoservicio
    , avg(ccheques_depositados) over ventana_2 as avg2_ccheques_depositados
    , avg(mcheques_depositados) over ventana_2 as avg2_mcheques_depositados
    , avg(ccheques_emitidos) over ventana_2 as avg2_ccheques_emitidos
    , avg(mcheques_emitidos) over ventana_2 as avg2_mcheques_emitidos
    , avg(ccheques_depositados_rechazados) over ventana_2 as avg2_ccheques_depositados_rechazados
    , avg(mcheques_depositados_rechazados) over ventana_2 as avg2_mcheques_depositados_rechazados
    , avg(ccheques_emitidos_rechazados) over ventana_2 as avg2_ccheques_emitidos_rechazados
    , avg(mcheques_emitidos_rechazados) over ventana_2 as avg2_mcheques_emitidos_rechazados
    , avg(ccallcenter_transacciones) over ventana_2 as avg2_ccallcenter_transacciones
    , avg(chomebanking_transacciones) over ventana_2 as avg2_chomebanking_transacciones
    , avg(ccajas_transacciones) over ventana_2 as avg2_ccajas_transacciones
    , avg(ccajas_consultas) over ventana_2 as avg2_ccajas_consultas
    , avg(ccajas_depositos) over ventana_2 as avg2_ccajas_depositos
    , avg(ccajas_extracciones) over ventana_2 as avg2_ccajas_extracciones
    , avg(ccajas_otras) over ventana_2 as avg2_ccajas_otras
    , avg(catm_trx) over ventana_2 as avg2_catm_trx
    , avg(matm) over ventana_2 as avg2_matm
    , avg(catm_trx_other) over ventana_2 as avg2_catm_trx_other
    , avg(matm_other) over ventana_2 as avg2_matm_other
    , avg(ctrx_quarter) over ventana_2 as avg2_ctrx_quarter
    , avg(Master_mfinanciacion_limite) over ventana_2 as avg2_Master_mfinanciacion_limite
    , avg(Master_msaldototal) over ventana_2 as avg2_Master_msaldototal
    , avg(Master_msaldopesos) over ventana_2 as avg2_Master_msaldopesos
    , avg(Master_msaldodolares) over ventana_2 as avg2_Master_msaldodolares
    , avg(Master_mconsumospesos) over ventana_2 as avg2_Master_mconsumospesos
    , avg(Master_mconsumosdolares) over ventana_2 as avg2_Master_mconsumosdolares
    , avg(Master_mlimitecompra) over ventana_2 as avg2_Master_mlimitecompra
    , avg(Master_madelantopesos) over ventana_2 as avg2_Master_madelantopesos
    , avg(Master_madelantodolares) over ventana_2 as avg2_Master_madelantodolares
    , avg(Master_mpagado) over ventana_2 as avg2_Master_mpagado
    , avg(Master_mpagospesos) over ventana_2 as avg2_Master_mpagospesos
    , avg(Master_mpagosdolares) over ventana_2 as avg2_Master_mpagosdolares
    , avg(Master_mconsumototal) over ventana_2 as avg2_Master_mconsumototal
    , avg(Master_cconsumos) over ventana_2 as avg2_Master_cconsumos
    , avg(Master_cadelantosefectivo) over ventana_2 as avg2_Master_cadelantosefectivo
    , avg(Master_mpagominimo) over ventana_2 as avg2_Master_mpagominimo
    , avg(Visa_mfinanciacion_limite) over ventana_2 as avg2_Visa_mfinanciacion_limite
    , avg(Visa_msaldototal) over ventana_2 as avg2_Visa_msaldototal
    , avg(Visa_msaldopesos) over ventana_2 as avg2_Visa_msaldopesos
    , avg(Visa_msaldodolares) over ventana_2 as avg2_Visa_msaldodolares
    , avg(Visa_mconsumospesos) over ventana_2 as avg2_Visa_mconsumospesos
    , avg(Visa_mconsumosdolares) over ventana_2 as avg2_Visa_mconsumosdolares
    , avg(Visa_mlimitecompra) over ventana_2 as avg2_Visa_mlimitecompra
    , avg(Visa_madelantopesos) over ventana_2 as avg2_Visa_madelantopesos
    , avg(Visa_madelantodolares) over ventana_2 as avg2_Visa_madelantodolares
    , avg(Visa_mpagado) over ventana_2 as avg2_Visa_mpagado
    , avg(Visa_mpagospesos) over ventana_2 as avg2_Visa_mpagospesos
    , avg(Visa_mpagosdolares) over ventana_2 as avg2_Visa_mpagosdolares
    , avg(Visa_mconsumototal) over ventana_2 as avg2_Visa_mconsumototal
    , avg(Visa_cconsumos) over ventana_2 as avg2_Visa_cconsumos
    , avg(Visa_cadelantosefectivo) over ventana_2 as avg2_Visa_cadelantosefectivo
    , avg(Visa_mpagominimo) over ventana_2 as avg2_Visa_mpagominimo
from competencia_02
window ventana_2 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

,Success


In [ ]:
#6. Promedios móviles con variables creadas.

In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
    , avg(map_margen) over ventana_2 as avg2_map_margen
    , avg(cc_ca) over ventana_2 as avg2_cc_ca
    , avg(ctarjeta_total) over ventana_2 as avg2_ctarjeta_total
    , avg(ctarjeta_total_transacciones) over ventana_2 as avg2_ctarjeta_total_transacciones
    , avg(mtarjeta_total_consumo) over ventana_2 as avg2_mtarjeta_total_consumo
    , avg(mplazo_fijo) over ventana_2 as avg2_mplazo_fijo
    , avg(minversion1) over ventana_2 as avg2_minversion1
    , avg(cseguro_todos) over ventana_2 as avg2_cseguro_todos
    , avg(mpayroll_todos) over ventana_2 as avg2_mpayroll_todos
    , avg(consumos_mas_DA) over ventana_2 as avg2_consumos_mas_DA
    , avg(ctarjeta_todas_debitos_automaticos) over ventana_2 as avg2_ctarjeta_todas_debitos_automaticos
    , avg(mtarjeta_todas_debitos_automaticos) over ventana_2 as avg2_mtarjeta_todas_debitos_automaticos
    , avg(ctarjeta_todas_descuentos) over ventana_2 as avg2_ctarjeta_todas_descuentos
    , avg(mtarjeta_todas_descuentos) over ventana_2 as avg2_mtarjeta_todas_descuentos
    , avg(comisiones) over ventana_2 as avg2_comisiones
    , avg(mcomisiones_suma) over ventana_2 as avg2_mcomisiones_suma
    , avg(ctransferencias) over ventana_2 as avg2_ctransferencias
    , avg(mtransferencias) over ventana_2 as avg2_mtransferencias
    , avg(tarjeta_msaldototal) over ventana_2 as avg2_tarjeta_msaldototal
    , avg(tarjetas_msaldopesos) over ventana_2 as avg2_tarjetas_msaldopesos
    , avg(tarjetas_msaldodolares) over ventana_2 as avg2_tarjetas_msaldodolares
    , avg(tarjetas_mconsumospesos) over ventana_2 as avg2_tarjetas_mconsumospesos
    , avg(tarjetas_mconsumosdolares) over ventana_2 as avg2_tarjetas_mconsumosdolares
    , avg(tarjetas_mlimitecompra) over ventana_2 as avg2_tarjetas_mlimitecompra
    , avg(tarjetas_madelantopesos) over ventana_2 as avg2_tarjetas_madelantopesos
    , avg(tarjetas_madelantodolares) over ventana_2 as avg2_tarjetas_madelantodolares
    , avg(tarjetas_mpagado) over ventana_2 as avg2_tarjetas_mpagado
    , avg(tarjetas_mpagospesos) over ventana_2 as avg2_tarjetas_mpagospesos
    , avg(tarjetas_mpagosdolares) over ventana_2 as avg2_tarjetas_mpagosdolares
    , avg(tarjetas_mconsumototal) over ventana_2 as avg2_tarjetas_mconsumototal
    , avg(tarjetas_cconsumos) over ventana_2 as avg2_tarjetas_cconsumos
    , avg(tarjetas_cadelantosefectivo) over ventana_2 as avg2_tarjetas_cadelantosefectivo
    , avg(tarjetas_mpagominimo) over ventana_2 as avg2_tarjetas_mpagominimo
    , avg(tarjetas_mfinanciacion_limite) over ventana_2 as avg2_tarjetas_mfinanciacion_limite
    , avg(matm_total) over ventana_2 as avg2_matm_total
from competencia_02
window ventana_2 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

## C. Creación de variables intermensuales (Data Drifting).

In [ ]:
#1. Slope con variables originales.

In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
    , regr_slope(mrentabilidad, cliente_antiguedad) over ventana_2 as slope_mrentabilidad
    , regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_2 as slope_mrentabilidad_annual
    , regr_slope(mcomisiones, cliente_antiguedad) over ventana_2 as slope_mcomisiones
    , regr_slope(mactivos_margen, cliente_antiguedad) over ventana_2 as slope_mactivos_margen
    , regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_2 as slope_mpasivos_margen
    , regr_slope(cproductos, cliente_antiguedad) over ventana_2 as slope_cproductos
    , regr_slope(tcuentas, cliente_antiguedad) over ventana_2 as slope_tcuentas
    , regr_slope(ccuenta_corriente, cliente_antiguedad) over ventana_2 as slope_ccuenta_corriente
    , regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_2 as slope_mcuenta_corriente_adicional
    , regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_2 as slope_mcuenta_corriente
    , regr_slope(ccaja_ahorro, cliente_antiguedad) over ventana_2 as slope_ccaja_ahorro
    , regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_2 as slope_mcaja_ahorro
    , regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_2 as slope_mcaja_ahorro_adicional
    , regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_2 as slope_mcaja_ahorro_dolares
    , regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_2 as slope_mcuentas_saldo
    , regr_slope(ctarjeta_debito, cliente_antiguedad) over ventana_2 as slope_ctarjeta_debito
    , regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_2 as slope_ctarjeta_debito_transacciones
    , regr_slope(mautoservicio, cliente_antiguedad) over ventana_2 as slope_mautoservicio
    , regr_slope(ctarjeta_visa, cliente_antiguedad) over ventana_2 as slope_ctarjeta_visa
    , regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_2 as slope_ctarjeta_visa_transacciones
    , regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_2 as slope_mtarjeta_visa_consumo
    , regr_slope(ctarjeta_master, cliente_antiguedad) over ventana_2 as slope_ctarjeta_master
    , regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana_2 as slope_ctarjeta_master_transacciones
    , regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_2 as slope_mtarjeta_master_consumo
    , regr_slope(cprestamos_prendarios, cliente_antiguedad) over ventana_2 as slope_cprestamos_prendarios
    , regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_2 as slope_mprestamos_prendarios
    , regr_slope(cprestamos_hipotecarios, cliente_antiguedad) over ventana_2 as slope_cprestamos_hipotecarios
    , regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_2 as slope_mprestamos_hipotecarios
    , regr_slope(cplazo_fijo, cliente_antiguedad) over ventana_2 as slope_cplazo_fijo
    , regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_2 as slope_mplazo_fijo_dolares
    , regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_2 as slope_mplazo_fijo_pesos
    , regr_slope(cinversion1, cliente_antiguedad) over ventana_2 as slope_cinversion1
    , regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_2 as slope_minversion1_pesos
    , regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_2 as slope_minversion1_dolares
    , regr_slope(cinversion2, cliente_antiguedad) over ventana_2 as slope_cinversion2
    , regr_slope(minversion2, cliente_antiguedad) over ventana_2 as slope_minversion2
    , regr_slope(cseguro_vida, cliente_antiguedad) over ventana_2 as slope_cseguro_vida
    , regr_slope(cseguro_auto, cliente_antiguedad) over ventana_2 as slope_cseguro_auto
    , regr_slope(cseguro_vivienda, cliente_antiguedad) over ventana_2 as slope_cseguro_vivienda
    , regr_slope(cseguro_accidentes_personales, cliente_antiguedad) over ventana_2 as slope_cseguro_accidentes_personales
    , regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_2 as slope_cpayroll_trx
    , regr_slope(mpayroll, cliente_antiguedad) over ventana_2 as slope_mpayroll
    , regr_slope(mpayroll2, cliente_antiguedad) over ventana_2 as slope_mpayroll2
    , regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_2 as slope_cpayroll2_trx
    , regr_slope(ccuenta_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_ccuenta_debitos_automaticos
    , regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_mcuenta_debitos_automaticos
    , regr_slope(ctarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_ctarjeta_visa_debitos_automaticos
    , regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_mttarjeta_visa_debitos_automaticos
    , regr_slope(ctarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_ctarjeta_master_debitos_automaticos
    , regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_mttarjeta_master_debitos_automaticos
    , regr_slope(cpagodeservicios, cliente_antiguedad) over ventana_2 as slope_cpagodeservicios
    , regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_2 as slope_mpagodeservicios
    , regr_slope(cpagomiscuentas, cliente_antiguedad) over ventana_2 as slope_cpagomiscuentas
    , regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_2 as slope_mpagomiscuentas
    , regr_slope(ccajeros_propios_descuentos, cliente_antiguedad) over ventana_2 as slope_ccajeros_propios_descuentos
    , regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_2 as slope_mcajeros_propios_descuentos
    , regr_slope(ctarjeta_visa_descuentos, cliente_antiguedad) over ventana_2 as slope_ctarjeta_visa_descuentos
    , regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_2 as slope_mtarjeta_visa_descuentos
    , regr_slope(ctarjeta_master_descuentos, cliente_antiguedad) over ventana_2 as slope_ctarjeta_master_descuentos
    , regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_2 as slope_mtarjeta_master_descuentos
    , regr_slope(ccomisiones_mantenimiento, cliente_antiguedad) over ventana_2 as slope_ccomisiones_mantenimiento
    , regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_2 as slope_mcomisiones_mantenimiento
    , regr_slope(ccomisiones_otras, cliente_antiguedad) over ventana_2 as slope_ccomisiones_otras
    , regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_2 as slope_mcomisiones_otras
    , regr_slope(cforex, cliente_antiguedad) over ventana_2 as slope_cforex
    , regr_slope(cforex_buy, cliente_antiguedad) over ventana_2 as slope_cforex_buy
    , regr_slope(mforex_buy, cliente_antiguedad) over ventana_2 as slope_mforex_buy
    , regr_slope(cforex_sell, cliente_antiguedad) over ventana_2 as slope_cforex_sell
    , regr_slope(mforex_sell, cliente_antiguedad) over ventana_2 as slope_mforex_sell
    , regr_slope(ctransferencias_recibidas, cliente_antiguedad) over ventana_2 as slope_ctransferencias_recibidas
    , regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_2 as slope_mtransferencias_recibidas
    , regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_2 as slope_ctransferencias_emitidas
    , regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_2 as slope_mtransferencias_emitidas
    , regr_slope(cextraccion_autoservicio, cliente_antiguedad) over ventana_2 as slope_cextraccion_autoservicio
    , regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_2 as slope_mextraccion_autoservicio
    , regr_slope(ccheques_depositados, cliente_antiguedad) over ventana_2 as slope_ccheques_depositados
    , regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_2 as slope_mcheques_depositados
    , regr_slope(ccheques_emitidos, cliente_antiguedad) over ventana_2 as slope_ccheques_emitidos
    , regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_2 as slope_mcheques_emitidos
    , regr_slope(ccheques_depositados_rechazados, cliente_antiguedad) over ventana_2 as slope_ccheques_depositados_rechazados
    , regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_2 as slope_mcheques_depositados_rechazados
    , regr_slope(ccheques_emitidos_rechazados, cliente_antiguedad) over ventana_2 as slope_ccheques_emitidos_rechazados
    , regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_2 as slope_mcheques_emitidos_rechazados
    , regr_slope(ccallcenter_transacciones, cliente_antiguedad) over ventana_2 as slope_ccallcenter_transacciones
    , regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_2 as slope_chomebanking_transacciones
    , regr_slope(ccajas_transacciones, cliente_antiguedad) over ventana_2 as slope_ccajas_transacciones
    , regr_slope(ccajas_consultas, cliente_antiguedad) over ventana_2 as slope_ccajas_consultas
    , regr_slope(ccajas_depositos, cliente_antiguedad) over ventana_2 as slope_ccajas_depositos
    , regr_slope(ccajas_extracciones, cliente_antiguedad) over ventana_2 as slope_ccajas_extracciones
    , regr_slope(ccajas_otras, cliente_antiguedad) over ventana_2 as slope_ccajas_otras
    , regr_slope(catm_trx, cliente_antiguedad) over ventana_2 as slope_catm_trx
    , regr_slope(matm, cliente_antiguedad) over ventana_2 as slope_matm
    , regr_slope(catm_trx_other, cliente_antiguedad) over ventana_2 as slope_catm_trx_other
    , regr_slope(matm_other, cliente_antiguedad) over ventana_2 as slope_matm_other
    , regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_2 as slope_ctrx_quarter
    , regr_slope(Master_mfinanciacion_limite, cliente_antiguedad) over ventana_2 as slope_Master_mfinanciacion_limite
    , regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_2 as slope_Master_msaldototal
    , regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_2 as slope_Master_msaldopesos
    , regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_2 as slope_Master_msaldodolares
    , regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_2 as slope_Master_mconsumospesos
    , regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_2 as slope_Master_mconsumosdolares
    , regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_2 as slope_Master_mlimitecompra
    , regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_2 as slope_Master_madelantopesos
    , regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_2 as slope_Master_madelantodolares
    , regr_slope(Master_mpagado, cliente_antiguedad) over ventana_2 as slope_Master_mpagado
    , regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_2 as slope_Master_mpagospesos
    , regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_2 as slope_Master_mpagosdolares
    , regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_2 as slope_Master_mconsumototal
    , regr_slope(Master_cconsumos, cliente_antiguedad) over ventana_2 as slope_Master_cconsumos
    , regr_slope(Master_cadelantosefectivo, cliente_antiguedad) over ventana_2 as slope_Master_cadelantosefectivo
    , regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_2 as slope_Master_mpagominimo
    , regr_slope(Visa_mfinanciacion_limite, cliente_antiguedad) over ventana_2 as slope_Visa_mfinanciacion_limite
    , regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_2 as slope_Visa_msaldototal
    , regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_2 as slope_Visa_msaldopesos
    , regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_2 as slope_Visa_msaldodolares
    , regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_2 as slope_Visa_mconsumospesos
    , regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_2 as slope_Visa_mconsumosdolares
    , regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_2 as slope_Visa_mlimitecompra
    , regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_2 as slope_Visa_madelantopesos
    , regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_2 as slope_Visa_madelantodolares
    , regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_2 as slope_Visa_mpagado
    , regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_2 as slope_Visa_mpagospesos
    , regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_2 as slope_Visa_mpagosdolares
    , regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_2 as slope_Visa_mconsumototal
    , regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana_2 as slope_Visa_cconsumos
    , regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_2 as slope_Visa_cadelantosefectivo
    , regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_2 as slope_Visa_mpagominimo
from competencia_02
window ventana_2 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

In [ ]:
#2. Slope con variables creadas.

In [ ]:
%%sql
create or replace table competencia_02 as
select
    *
    , regr_slope(map_margen, cliente_antiguedad) over ventana_2 as slope_map_margen
    , regr_slope(cc_ca, cliente_antiguedad) over ventana_2 as slope_cc_ca
    , regr_slope(ctarjeta_total, cliente_antiguedad) over ventana_2 as slope_ctarjeta_total
    , regr_slope(ctarjeta_total_transacciones, cliente_antiguedad) over ventana_2 as slope_ctarjeta_total_transacciones
    , regr_slope(mtarjeta_total_consumo, cliente_antiguedad) over ventana_2 as slope_mtarjeta_total_consumo
    , regr_slope(mplazo_fijo, cliente_antiguedad) over ventana_2 as slope_mplazo_fijo
    , regr_slope(minversion1, cliente_antiguedad) over ventana_2 as slope_minversion1
    , regr_slope(cseguro_todos, cliente_antiguedad) over ventana_2 as slope_cseguro_todos
    , regr_slope(mpayroll_todos, cliente_antiguedad) over ventana_2 as slope_mpayroll_todos
    , regr_slope(consumos_mas_DA, cliente_antiguedad) over ventana_2 as slope_consumos_mas_DA
    , regr_slope(ctarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_ctarjeta_todas_debitos_automaticos
    , regr_slope(mtarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_mtarjeta_todas_debitos_automaticos
    , regr_slope(ctarjeta_todas_descuentos, cliente_antiguedad) over ventana_2 as slope_ctarjeta_todas_descuentos
    , regr_slope(mtarjeta_todas_descuentos, cliente_antiguedad) over ventana_2 as slope_mtarjeta_todas_descuentos
    , regr_slope(comisiones, cliente_antiguedad) over ventana_2 as slope_comisiones
    , regr_slope(mcomisiones_suma, cliente_antiguedad) over ventana_2 as slope_mcomisiones_suma
    , regr_slope(ctransferencias, cliente_antiguedad) over ventana_2 as slope_ctransferencias
    , regr_slope(mtransferencias, cliente_antiguedad) over ventana_2 as slope_mtransferencias
    , regr_slope(tarjeta_msaldototal, cliente_antiguedad) over ventana_2 as slope_tarjeta_msaldototal
    , regr_slope(tarjetas_msaldopesos, cliente_antiguedad) over ventana_2 as slope_tarjetas_msaldopesos
    , regr_slope(tarjetas_msaldodolares, cliente_antiguedad) over ventana_2 as slope_tarjetas_msaldodolares
    , regr_slope(tarjetas_mconsumospesos, cliente_antiguedad) over ventana_2 as slope_tarjetas_mconsumospesos
    , regr_slope(tarjetas_mconsumosdolares, cliente_antiguedad) over ventana_2 as slope_tarjetas_mconsumosdolares
    , regr_slope(tarjetas_mlimitecompra, cliente_antiguedad) over ventana_2 as slope_tarjetas_mlimitecompra
    , regr_slope(tarjetas_madelantopesos, cliente_antiguedad) over ventana_2 as slope_tarjetas_madelantopesos
    , regr_slope(tarjetas_madelantodolares, cliente_antiguedad) over ventana_2 as slope_tarjetas_madelantodolares
    , regr_slope(tarjetas_mpagado, cliente_antiguedad) over ventana_2 as slope_tarjetas_mpagado
    , regr_slope(tarjetas_mpagospesos, cliente_antiguedad) over ventana_2 as slope_tarjetas_mpagospesos
    , regr_slope(tarjetas_mpagosdolares, cliente_antiguedad) over ventana_2 as slope_tarjetas_mpagosdolares
    , regr_slope(tarjetas_mconsumototal, cliente_antiguedad) over ventana_2 as slope_tarjetas_mconsumototal
    , regr_slope(tarjetas_cconsumos, cliente_antiguedad) over ventana_2 as slope_tarjetas_cconsumos
    , regr_slope(tarjetas_cadelantosefectivo, cliente_antiguedad) over ventana_2 as slope_tarjetas_cadelantosefectivo
    , regr_slope(tarjetas_mpagominimo, cliente_antiguedad) over ventana_2 as slope_tarjetas_mpagominimo
    , regr_slope(tarjetas_mfinanciacion_limite, cliente_antiguedad) over ventana_2 as slope_tarjetas_mfinanciacion_limite
    , regr_slope(matm_total, cliente_antiguedad) over ventana_2 as slope_matm_total
    , regr_slope(avg2_mrentabilidad, cliente_antiguedad) over ventana_2 as slope_avg2_mrentabilidad
    , regr_slope(avg2_mrentabilidad_annual, cliente_antiguedad) over ventana_2 as slope_avg2_mrentabilidad_annual
    , regr_slope(avg2_mcomisiones, cliente_antiguedad) over ventana_2 as slope_avg2_mcomisiones
    , regr_slope(avg2_mactivos_margen, cliente_antiguedad) over ventana_2 as slope_avg2_mactivos_margen
    , regr_slope(avg2_mpasivos_margen, cliente_antiguedad) over ventana_2 as slope_avg2_mpasivos_margen
    , regr_slope(avg2_cproductos, cliente_antiguedad) over ventana_2 as slope_avg2_cproductos
    , regr_slope(avg2_tcuentas, cliente_antiguedad) over ventana_2 as slope_avg2_tcuentas
    , regr_slope(avg2_ccuenta_corriente, cliente_antiguedad) over ventana_2 as slope_avg2_ccuenta_corriente
    , regr_slope(avg2_mcuenta_corriente_adicional, cliente_antiguedad) over ventana_2 as slope_avg2_mcuenta_corriente_adicional
    , regr_slope(avg2_mcuenta_corriente, cliente_antiguedad) over ventana_2 as slope_avg2_mcuenta_corriente
    , regr_slope(avg2_ccaja_ahorro, cliente_antiguedad) over ventana_2 as slope_avg2_ccaja_ahorro
    , regr_slope(avg2_mcaja_ahorro, cliente_antiguedad) over ventana_2 as slope_avg2_mcaja_ahorro
    , regr_slope(avg2_mcaja_ahorro_adicional, cliente_antiguedad) over ventana_2 as slope_avg2_mcaja_ahorro_adicional
    , regr_slope(avg2_mcaja_ahorro_dolares, cliente_antiguedad) over ventana_2 as slope_avg2_mcaja_ahorro_dolares
    , regr_slope(avg2_mcuentas_saldo, cliente_antiguedad) over ventana_2 as slope_avg2_mcuentas_saldo
    , regr_slope(avg2_ctarjeta_debito, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_debito
    , regr_slope(avg2_ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_debito_transacciones
    , regr_slope(avg2_mautoservicio, cliente_antiguedad) over ventana_2 as slope_avg2_mautoservicio
    , regr_slope(avg2_ctarjeta_visa, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_visa
    , regr_slope(avg2_ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_visa_transacciones
    , regr_slope(avg2_mtarjeta_visa_consumo, cliente_antiguedad) over ventana_2 as slope_avg2_mtarjeta_visa_consumo
    , regr_slope(avg2_ctarjeta_master, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_master
    , regr_slope(avg2_ctarjeta_master_transacciones, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_master_transacciones
    , regr_slope(avg2_mtarjeta_master_consumo, cliente_antiguedad) over ventana_2 as slope_avg2_mtarjeta_master_consumo
    , regr_slope(avg2_cprestamos_prendarios, cliente_antiguedad) over ventana_2 as slope_avg2_cprestamos_prendarios
    , regr_slope(avg2_mprestamos_prendarios, cliente_antiguedad) over ventana_2 as slope_avg2_mprestamos_prendarios
    , regr_slope(avg2_cprestamos_hipotecarios, cliente_antiguedad) over ventana_2 as slope_avg2_cprestamos_hipotecarios
    , regr_slope(avg2_mprestamos_hipotecarios, cliente_antiguedad) over ventana_2 as slope_avg2_mprestamos_hipotecarios
    , regr_slope(avg2_cplazo_fijo, cliente_antiguedad) over ventana_2 as slope_avg2_cplazo_fijo
    , regr_slope(avg2_mplazo_fijo_dolares, cliente_antiguedad) over ventana_2 as slope_avg2_mplazo_fijo_dolares
    , regr_slope(avg2_mplazo_fijo_pesos, cliente_antiguedad) over ventana_2 as slope_avg2_mplazo_fijo_pesos
    , regr_slope(avg2_cinversion1, cliente_antiguedad) over ventana_2 as slope_avg2_cinversion1
    , regr_slope(avg2_minversion1_pesos, cliente_antiguedad) over ventana_2 as slope_avg2_minversion1_pesos
    , regr_slope(avg2_minversion1_dolares, cliente_antiguedad) over ventana_2 as slope_avg2_minversion1_dolares
    , regr_slope(avg2_cinversion2, cliente_antiguedad) over ventana_2 as slope_avg2_cinversion2
    , regr_slope(avg2_minversion2, cliente_antiguedad) over ventana_2 as slope_avg2_minversion2
    , regr_slope(avg2_cseguro_vida, cliente_antiguedad) over ventana_2 as slope_avg2_cseguro_vida
    , regr_slope(avg2_cseguro_auto, cliente_antiguedad) over ventana_2 as slope_avg2_cseguro_auto
    , regr_slope(avg2_cseguro_vivienda, cliente_antiguedad) over ventana_2 as slope_avg2_cseguro_vivienda
    , regr_slope(avg2_cseguro_accidentes_personales, cliente_antiguedad) over ventana_2 as slope_avg2_cseguro_accidentes_personales
    , regr_slope(avg2_cpayroll_trx, cliente_antiguedad) over ventana_2 as slope_avg2_cpayroll_trx
    , regr_slope(avg2_mpayroll, cliente_antiguedad) over ventana_2 as slope_avg2_mpayroll
    , regr_slope(avg2_mpayroll2, cliente_antiguedad) over ventana_2 as slope_avg2_mpayroll2
    , regr_slope(avg2_cpayroll2_trx, cliente_antiguedad) over ventana_2 as slope_avg2_cpayroll2_trx
    , regr_slope(avg2_ccuenta_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_avg2_ccuenta_debitos_automaticos
    , regr_slope(avg2_mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_avg2_mcuenta_debitos_automaticos
    , regr_slope(avg2_ctarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_visa_debitos_automaticos
    , regr_slope(avg2_mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_avg2_mttarjeta_visa_debitos_automaticos
    , regr_slope(avg2_ctarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_master_debitos_automaticos
    , regr_slope(avg2_mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_avg2_mttarjeta_master_debitos_automaticos
    , regr_slope(avg2_cpagodeservicios, cliente_antiguedad) over ventana_2 as slope_avg2_cpagodeservicios
    , regr_slope(avg2_mpagodeservicios, cliente_antiguedad) over ventana_2 as slope_avg2_mpagodeservicios
    , regr_slope(avg2_cpagomiscuentas, cliente_antiguedad) over ventana_2 as slope_avg2_cpagomiscuentas
    , regr_slope(avg2_mpagomiscuentas, cliente_antiguedad) over ventana_2 as slope_avg2_mpagomiscuentas
    , regr_slope(avg2_ccajeros_propios_descuentos, cliente_antiguedad) over ventana_2 as slope_avg2_ccajeros_propios_descuentos
    , regr_slope(avg2_mcajeros_propios_descuentos, cliente_antiguedad) over ventana_2 as slope_avg2_mcajeros_propios_descuentos
    , regr_slope(avg2_ctarjeta_visa_descuentos, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_visa_descuentos
    , regr_slope(avg2_mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_2 as slope_avg2_mtarjeta_visa_descuentos
    , regr_slope(avg2_ctarjeta_master_descuentos, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_master_descuentos
    , regr_slope(avg2_mtarjeta_master_descuentos, cliente_antiguedad) over ventana_2 as slope_avg2_mtarjeta_master_descuentos
    , regr_slope(avg2_ccomisiones_mantenimiento, cliente_antiguedad) over ventana_2 as slope_avg2_ccomisiones_mantenimiento
    , regr_slope(avg2_mcomisiones_mantenimiento, cliente_antiguedad) over ventana_2 as slope_avg2_mcomisiones_mantenimiento
    , regr_slope(avg2_ccomisiones_otras, cliente_antiguedad) over ventana_2 as slope_avg2_ccomisiones_otras
    , regr_slope(avg2_mcomisiones_otras, cliente_antiguedad) over ventana_2 as slope_avg2_mcomisiones_otras
    , regr_slope(avg2_cforex, cliente_antiguedad) over ventana_2 as slope_avg2_cforex
    , regr_slope(avg2_cforex_buy, cliente_antiguedad) over ventana_2 as slope_avg2_cforex_buy
    , regr_slope(avg2_mforex_buy, cliente_antiguedad) over ventana_2 as slope_avg2_mforex_buy
    , regr_slope(avg2_cforex_sell, cliente_antiguedad) over ventana_2 as slope_avg2_cforex_sell
    , regr_slope(avg2_mforex_sell, cliente_antiguedad) over ventana_2 as slope_avg2_mforex_sell
    , regr_slope(avg2_ctransferencias_recibidas, cliente_antiguedad) over ventana_2 as slope_avg2_ctransferencias_recibidas
    , regr_slope(avg2_mtransferencias_recibidas, cliente_antiguedad) over ventana_2 as slope_avg2_mtransferencias_recibidas
    , regr_slope(avg2_ctransferencias_emitidas, cliente_antiguedad) over ventana_2 as slope_avg2_ctransferencias_emitidas
    , regr_slope(avg2_mtransferencias_emitidas, cliente_antiguedad) over ventana_2 as slope_avg2_mtransferencias_emitidas
    , regr_slope(avg2_cextraccion_autoservicio, cliente_antiguedad) over ventana_2 as slope_avg2_cextraccion_autoservicio
    , regr_slope(avg2_mextraccion_autoservicio, cliente_antiguedad) over ventana_2 as slope_avg2_mextraccion_autoservicio
    , regr_slope(avg2_ccheques_depositados, cliente_antiguedad) over ventana_2 as slope_avg2_ccheques_depositados
    , regr_slope(avg2_mcheques_depositados, cliente_antiguedad) over ventana_2 as slope_avg2_mcheques_depositados
    , regr_slope(avg2_ccheques_emitidos, cliente_antiguedad) over ventana_2 as slope_avg2_ccheques_emitidos
    , regr_slope(avg2_mcheques_emitidos, cliente_antiguedad) over ventana_2 as slope_avg2_mcheques_emitidos
    , regr_slope(avg2_ccheques_depositados_rechazados, cliente_antiguedad) over ventana_2 as slope_avg2_ccheques_depositados_rechazados
    , regr_slope(avg2_mcheques_depositados_rechazados, cliente_antiguedad) over ventana_2 as slope_avg2_mcheques_depositados_rechazados
    , regr_slope(avg2_ccheques_emitidos_rechazados, cliente_antiguedad) over ventana_2 as slope_avg2_ccheques_emitidos_rechazados
    , regr_slope(avg2_mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_2 as slope_avg2_mcheques_emitidos_rechazados
    , regr_slope(avg2_ccallcenter_transacciones, cliente_antiguedad) over ventana_2 as slope_avg2_ccallcenter_transacciones
    , regr_slope(avg2_chomebanking_transacciones, cliente_antiguedad) over ventana_2 as slope_avg2_chomebanking_transacciones
    , regr_slope(avg2_ccajas_transacciones, cliente_antiguedad) over ventana_2 as slope_avg2_ccajas_transacciones
    , regr_slope(avg2_ccajas_consultas, cliente_antiguedad) over ventana_2 as slope_avg2_ccajas_consultas
    , regr_slope(avg2_ccajas_depositos, cliente_antiguedad) over ventana_2 as slope_avg2_ccajas_depositos
    , regr_slope(avg2_ccajas_extracciones, cliente_antiguedad) over ventana_2 as slope_avg2_ccajas_extracciones
    , regr_slope(avg2_ccajas_otras, cliente_antiguedad) over ventana_2 as slope_avg2_ccajas_otras
    , regr_slope(avg2_catm_trx, cliente_antiguedad) over ventana_2 as slope_avg2_catm_trx
    , regr_slope(avg2_matm, cliente_antiguedad) over ventana_2 as slope_avg2_matm
    , regr_slope(avg2_catm_trx_other, cliente_antiguedad) over ventana_2 as slope_avg2_catm_trx_other
    , regr_slope(avg2_matm_other, cliente_antiguedad) over ventana_2 as slope_avg2_matm_other
    , regr_slope(avg2_ctrx_quarter, cliente_antiguedad) over ventana_2 as slope_avg2_ctrx_quarter
    , regr_slope(avg2_Master_mfinanciacion_limite, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mfinanciacion_limite
    , regr_slope(avg2_Master_msaldototal, cliente_antiguedad) over ventana_2 as slope_avg2_Master_msaldototal
    , regr_slope(avg2_Master_msaldopesos, cliente_antiguedad) over ventana_2 as slope_avg2_Master_msaldopesos
    , regr_slope(avg2_Master_msaldodolares, cliente_antiguedad) over ventana_2 as slope_avg2_Master_msaldodolares
    , regr_slope(avg2_Master_mconsumospesos, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mconsumospesos
    , regr_slope(avg2_Master_mconsumosdolares, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mconsumosdolares
    , regr_slope(avg2_Master_mlimitecompra, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mlimitecompra
    , regr_slope(avg2_Master_madelantopesos, cliente_antiguedad) over ventana_2 as slope_avg2_Master_madelantopesos
    , regr_slope(avg2_Master_madelantodolares, cliente_antiguedad) over ventana_2 as slope_avg2_Master_madelantodolares
    , regr_slope(avg2_Master_mpagado, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mpagado
    , regr_slope(avg2_Master_mpagospesos, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mpagospesos
    , regr_slope(avg2_Master_mpagosdolares, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mpagosdolares
    , regr_slope(avg2_Master_mconsumototal, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mconsumototal
    , regr_slope(avg2_Master_cconsumos, cliente_antiguedad) over ventana_2 as slope_avg2_Master_cconsumos
    , regr_slope(avg2_Master_cadelantosefectivo, cliente_antiguedad) over ventana_2 as slope_avg2_Master_cadelantosefectivo
    , regr_slope(avg2_Master_mpagominimo, cliente_antiguedad) over ventana_2 as slope_avg2_Master_mpagominimo
    , regr_slope(avg2_Visa_mfinanciacion_limite, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mfinanciacion_limite
    , regr_slope(avg2_Visa_msaldototal, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_msaldototal
    , regr_slope(avg2_Visa_msaldopesos, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_msaldopesos
    , regr_slope(avg2_Visa_msaldodolares, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_msaldodolares
    , regr_slope(avg2_Visa_mconsumospesos, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mconsumospesos
    , regr_slope(avg2_Visa_mconsumosdolares, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mconsumosdolares
    , regr_slope(avg2_Visa_mlimitecompra, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mlimitecompra
    , regr_slope(avg2_Visa_madelantopesos, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_madelantopesos
    , regr_slope(avg2_Visa_madelantodolares, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_madelantodolares
    , regr_slope(avg2_Visa_mpagado, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mpagado
    , regr_slope(avg2_Visa_mpagospesos, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mpagospesos
    , regr_slope(avg2_Visa_mpagosdolares, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mpagosdolares
    , regr_slope(avg2_Visa_mconsumototal, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mconsumototal
    , regr_slope(avg2_Visa_cconsumos, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_cconsumos
    , regr_slope(avg2_Visa_cadelantosefectivo, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_cadelantosefectivo
    , regr_slope(avg2_Visa_mpagominimo, cliente_antiguedad) over ventana_2 as slope_avg2_Visa_mpagominimo
    , regr_slope(avg2_map_margen, cliente_antiguedad) over ventana_2 as slope_avg2_map_margen
    , regr_slope(avg2_cc_ca, cliente_antiguedad) over ventana_2 as slope_avg2_cc_ca
    , regr_slope(avg2_ctarjeta_total, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_total
    , regr_slope(avg2_ctarjeta_total_transacciones, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_total_transacciones
    , regr_slope(avg2_mtarjeta_total_consumo, cliente_antiguedad) over ventana_2 as slope_avg2_mtarjeta_total_consumo
    , regr_slope(avg2_mplazo_fijo, cliente_antiguedad) over ventana_2 as slope_avg2_mplazo_fijo
    , regr_slope(avg2_minversion1, cliente_antiguedad) over ventana_2 as slope_avg2_minversion1
    , regr_slope(avg2_cseguro_todos, cliente_antiguedad) over ventana_2 as slope_avg2_cseguro_todos
    , regr_slope(avg2_mpayroll_todos, cliente_antiguedad) over ventana_2 as slope_avg2_mpayroll_todos
    , regr_slope(avg2_consumos_mas_DA, cliente_antiguedad) over ventana_2 as slope_avg2_consumos_mas_DA
    , regr_slope(avg2_ctarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_todas_debitos_automaticos
    , regr_slope(avg2_mtarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_2 as slope_avg2_mtarjeta_todas_debitos_automaticos
    , regr_slope(avg2_ctarjeta_todas_descuentos, cliente_antiguedad) over ventana_2 as slope_avg2_ctarjeta_todas_descuentos
    , regr_slope(avg2_mtarjeta_todas_descuentos, cliente_antiguedad) over ventana_2 as slope_avg2_mtarjeta_todas_descuentos
    , regr_slope(avg2_comisiones, cliente_antiguedad) over ventana_2 as slope_avg2_comisiones
    , regr_slope(avg2_mcomisiones_suma, cliente_antiguedad) over ventana_2 as slope_avg2_mcomisiones_suma
    , regr_slope(avg2_ctransferencias, cliente_antiguedad) over ventana_2 as slope_avg2_ctransferencias
    , regr_slope(avg2_mtransferencias, cliente_antiguedad) over ventana_2 as slope_avg2_mtransferencias
    , regr_slope(avg2_tarjeta_msaldototal, cliente_antiguedad) over ventana_2 as slope_avg2_tarjeta_msaldototal
    , regr_slope(avg2_tarjetas_msaldopesos, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_msaldopesos
    , regr_slope(avg2_tarjetas_msaldodolares, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_msaldodolares
    , regr_slope(avg2_tarjetas_mconsumospesos, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mconsumospesos
    , regr_slope(avg2_tarjetas_mconsumosdolares, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mconsumosdolares
    , regr_slope(avg2_tarjetas_mlimitecompra, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mlimitecompra
    , regr_slope(avg2_tarjetas_madelantopesos, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_madelantopesos
    , regr_slope(avg2_tarjetas_madelantodolares, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_madelantodolares
    , regr_slope(avg2_tarjetas_mpagado, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mpagado
    , regr_slope(avg2_tarjetas_mpagospesos, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mpagospesos
    , regr_slope(avg2_tarjetas_mpagosdolares, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mpagosdolares
    , regr_slope(avg2_tarjetas_mconsumototal, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mconsumototal
    , regr_slope(avg2_tarjetas_cconsumos, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_cconsumos
    , regr_slope(avg2_tarjetas_cadelantosefectivo, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_cadelantosefectivo
    , regr_slope(avg2_tarjetas_mpagominimo, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mpagominimo
    , regr_slope(avg2_tarjetas_mfinanciacion_limite, cliente_antiguedad) over ventana_2 as slope_avg2_tarjetas_mfinanciacion_limite
    , regr_slope(avg2_matm_total, cliente_antiguedad) over ventana_2 as slope_avg2_matm_total
from competencia_02
window ventana_2 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

## D. Exportación.

In [ ]:
%%sql
COPY competencia_02 TO '{{dataset_output}}' (FORMAT PARQUET);

,Success
